## Generate the "observed" dataset

Here we will generate training, validation, and test sets that will be used as our "observed" dataset. These will consist of spectra generated using The Payne with noise added afterwards. These spectra are going to contain the entire line list, as opposed to the "synthetic" dataset, which will have particular lines masked out (ie. set to the continuum value).

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
%matplotlib inline 

from The_Payne.utils import read_in_neural_network

import torch
from torch.autograd import Variable

import sys
sys.path.append('../')
from network import build_emulator

data_dir = '../data/'

np.random.seed(1)

Build The Payne network

In [2]:
# Load the Payne pre-trained weights
emulator_coeffs = read_in_neural_network()

# Build emulator and assign layer weights
emulator, y_min, y_max = build_emulator(emulator_coeffs=emulator_coeffs, use_cuda=False)

Load the Payne labels that were used to train the model initially

In [3]:
# Order:

# [Teff, Logg, Vturb [km/s],
# [C/H], [N/H], [O/H], [Na/H], [Mg/H],\
# [Al/H], [Si/H], [P/H], [S/H], [K/H],\
# [Ca/H], [Ti/H], [V/H], [Cr/H], [Mn/H],\
# [Fe/H], [Co/H], [Ni/H], [Cu/H], [Ge/H],\
# C12/C13, Vmacro [km/s]
labels_payne = np.load(data_dir+'mock_all_spectra_no_noise_resample_prior_large.npz')['labels'].T

Generate the labels for our training, validation, and test sets. To do this, we will take the initial labels that were loaded above and perturb them slightly to add some randomness to our datasets. Ideally, we would draw these labels randomly from a large distribution, however, the Payne can only produce spectra accurately that are within the limits of its training set. Therefore, we must stay close to this initial distribution.

In [4]:
# The number of Payne spectra to create for each dataset
num_tr = 50000
num_val = 2000
num_te = 2000

# Perturb the payne labels within a range to create our training, cv, and test sets.
# These perturbations are in the same order as the labels.
perturbations = [100., 0.1, 0.2, *np.repeat(0.1, 20), 5., 2.]

# Create sets by randomly selecting from original Payne training set
payne_tr_labels = np.copy(labels_payne[np.random.randint(len(labels_payne), size=num_tr)])
payne_val_labels = np.copy(labels_payne[np.random.randint(len(labels_payne), size=num_val)])
payne_te_labels = np.copy(labels_payne[np.random.randint(len(labels_payne), size=num_te)])

# Apply random perturbations within the above limits
payne_tr_labels += np.array([np.random.uniform(-1*p, p, size=num_tr) for p in perturbations]).T
payne_val_labels += np.array([np.random.uniform(-1*p, p, size=num_val) for p in perturbations]).T
payne_te_labels += np.array([np.random.uniform(-1*p, p, size=num_te) for p in perturbations]).T

# Correct the minimum Vturb, C12/C13, and Vmacro values
for i in [2,23,24]:
    payne_tr_labels[payne_tr_labels[:,i]<np.min(labels_payne[:,i]),i] = np.min(labels_payne[:,i])
    payne_val_labels[payne_val_labels[:,i]<np.min(labels_payne[:,i]),i] = np.min(labels_payne[:,i])
    payne_te_labels[payne_te_labels[:,i]<np.min(labels_payne[:,i]),i] = np.min(labels_payne[:,i])
    
# Set Vmacro for the first 1000 samples in the validation and test sets to 15km/s.
# This will allow us to test our split latent-space method by generating matching spectra in the 
# "synthetic" domain to do direct comparisons with.
payne_val_labels[:1000,24] = 15.
payne_te_labels[:1000,24] = 15.

Now we will use the Payne model to produce the spectra from the labels created above.

In [5]:
def predict_spectra(emulator, y, x_min, x_max):
    # Turn labels into pytorch variable
    y = Variable(torch.Tensor(y.astype(np.float32)))
    # Scale the labels the same as it was done during the training of the Payne
    scaled_labels = (y-y_min)/(y_max-y_min) - 0.5
    # Return predicted spectra
    return emulator(scaled_labels).data.numpy()

payne_tr_specs = predict_spectra(emulator, payne_tr_labels, y_min, y_max)
payne_val_specs = predict_spectra(emulator, payne_val_labels, y_min, y_max)
payne_te_specs = predict_spectra(emulator, payne_te_labels, y_min, y_max)

Add some noise to each spectrum.

In [6]:
# Approximiate limits to the noise added. A noise factor or 0.05 will give that 
# particular pixel a S/N of 1/0.05 = 20.
max_noise = 0.05
min_noise = 0.001

# Create a noise factor for each spectrum
noise_factor_tr = np.random.uniform(low=min_noise, high=max_noise, size=(len(payne_tr_specs),1))
noise_factor_val = np.random.uniform(low=min_noise, high=max_noise, size=(len(payne_val_specs),1))
noise_factor_te = np.random.uniform(low=min_noise, high=max_noise, size=(len(payne_te_specs),1))

# Generate noise spectra that will be added to the spectra
spec_err_tr = noise_factor_tr*np.random.uniform(low=0.1, high=1., size=payne_tr_specs.shape)
spec_err_val = noise_factor_val*np.random.uniform(low=0.1, high=1., size=payne_val_specs.shape)
spec_err_te = noise_factor_te*np.random.uniform(low=0.1, high=1., size=payne_te_specs.shape)

# Add noise
payne_tr_specs += spec_err_tr
payne_val_specs += spec_err_val
payne_te_specs += spec_err_te

To give an idea of the approximate S/N limits, we will calculate the S/N for the cross-validation set.

In [7]:
snr_val = np.mean(payne_val_specs/spec_err_val, axis=1)
print('The S/N of our observed dataset ranges from approximately %0.0f to %0.0f.' % (np.min(snr_val), np.max(snr_val)))

The S/N of our observed dataset ranges from approximately 49 to 2518.


Save data to an hdf5 file.

In [8]:
savename_synth = data_dir+'PAYNE_NL.h5'

with h5py.File(savename_synth, "w") as f:    
    
    # Datasets for h5 file
    spec_tr_ds = f.create_dataset('PAYNE spectrum train', 
                                  payne_tr_specs.shape, dtype="f")
    spec_val_ds = f.create_dataset('PAYNE spectrum val', 
                                  payne_val_specs.shape, dtype="f")
    spec_te_ds = f.create_dataset('PAYNE spectrum test', 
                                  payne_te_specs.shape, dtype="f")
    spec_err_tr_ds = f.create_dataset('PAYNE error_spectrum train', 
                                  spec_err_tr.shape, dtype="f")
    spec_err_val_ds = f.create_dataset('PAYNE error_spectrum val', 
                                  spec_err_val.shape, dtype="f")
    spec_err_te_ds = f.create_dataset('PAYNE error_spectrum test', 
                                  spec_err_te.shape, dtype="f")
    labels_tr_ds = f.create_dataset('PAYNE labels train', 
                                  payne_tr_labels.shape, dtype="f")
    labels_val_ds = f.create_dataset('PAYNE labels val', 
                                  payne_val_labels.shape, dtype="f")
    labels_te_ds = f.create_dataset('PAYNE labels test', 
                                  payne_te_labels.shape, dtype="f")
    
    # Save data
    spec_tr_ds[:] = payne_tr_specs
    spec_val_ds[:] = payne_val_specs
    spec_te_ds[:] = payne_te_specs
    spec_err_tr_ds[:] = spec_err_tr
    spec_err_val_ds[:] = spec_err_val
    spec_err_te_ds[:] = spec_err_te
    labels_tr_ds[:] = payne_tr_labels
    labels_val_ds[:] = payne_val_labels
    labels_te_ds[:] = payne_te_labels
            
print('finished.')

finished.


Lastly, collect the mean and standard deviation of the synthetic spectra for normalizing the inputs to the Cycle-GAN.

In [9]:
spectra_payne = np.load(data_dir+'mock_all_spectra_no_noise_resample_prior_large.npz')['spectra']
np.save(data_dir+'mean_and_std_PAYNE_specs.npy', np.array([np.mean(spectra_payne),np.std(spectra_payne)]))